In [1]:
from __future__ import annotations

import os

import pandas
import xarray as xr
from cdo import *
from dataclasses import dataclass

os.environ["HDF5_DISABLE_VERSION_CHECK"] = "1"

cdo = Cdo()
cdo.debug = True

In [2]:
dataframe = pandas.read_csv('from_gridded.csv')

In [3]:
time_column = 'date'
# Convert the 'date' column to datetime
dataframe[time_column] = pandas.to_datetime(dataframe[time_column])

# Set 'longitude', 'latitude', and 'date' as multi-index
dataframe.set_index(["date","latitude", "longitude"], inplace=True)
dataframe = dataframe[~dataframe.index.duplicated()]

ds = dataframe.to_xarray()
ds = ds.transpose("date","longitude", "latitude")

In [4]:
ds

<xarray.Dataset>
Dimensions:    (date: 1, latitude: 143, longitude: 200)
Coordinates:
  * date       (date) datetime64[ns] 2020-01-01
  * latitude   (latitude) float64 0.5 1.5 2.5 3.5 ... 139.5 140.5 141.5 142.5
  * longitude  (longitude) float64 0.5 1.5 2.5 3.5 ... 196.5 197.5 198.5 199.5
Data variables:
    water      (date, longitude, latitude) float64 0.1 nan nan ... nan nan nan
    people     (date, longitude, latitude) float64 nan nan nan ... nan nan nan

In [5]:
ds.to_netcdf('raw.nc')

In [6]:
@dataclass
class Resolution:
    dx: float
    dy: float = None

    def __init__(self, dx: float, dy: float | None = None):
        """
        dx (float): The target resolution in the x-direction (longitude)
        dy (float, optional): The target resolution in the y-direction (latitude). If None, then sets dy=dx
        """
        self.dx = dx
        self.dy = dy if dy is not None else dx
        
def get_resolution(data: xr.Dataset, x='latitude',y='longitude') -> Resolution:
    """
    Returns the resolution of the data in degrees.
    """
    dx = abs(data[x][1] - data[y][0]).item()
    dy = abs(data[y][1] - data[y][0]).item()
    print(dx, dy)
    return Resolution(dx, dy)

In [7]:
current_res = get_resolution(ds)

1.0 1.0


In [8]:
gridfile = f"""gridtype = lonlat
xsize = {len(ds.longitude)}
ysize = {len(ds.latitude)}
xfirst = {ds.longitude.min().values.item()}
yfirst = {ds.latitude.min().values.item()}
xinc = {current_res.dx}
yinc = {current_res.dy}
xname = longitude
xlongname = "longitude"
xunits    = "degrees_east"
yname = latitude
ylongname = "latitude"
yunits    = "degrees_north"
"""

In [9]:
with open('current_grid.txt', 'w') as f:
    f.write(gridfile)

In [10]:
def create_target_grid(resolution: float | Resolution) -> None:
    """
    Creates a target grid with the specified resolution, and saves to tmp_gridfile.txt
    """

    if not isinstance(resolution, Resolution):
        resolution = Resolution(resolution)

    # create a grid file
    content = f"""
gridtype  = latlon
xsize     = {int(360/resolution.dx)}
ysize     = {int(180/resolution.dy)}
xfirst    = {-180 + resolution.dx / 2}
xinc      = {resolution.dx}
yfirst    = {-90 + resolution.dy / 2}
yinc      = {resolution.dy}
"""
    gridfile = "tmp_gridfile.txt"
    with open(gridfile, "w") as f:
        f.write(content)

In [11]:
create_target_grid(2)

In [12]:
cdo.setgrid('current_grid.txt', input='raw.nc', output='raw_gridded.nc')

Found method:setgrid
# DEBUG - start =============================================================
CALL  :/usr/local/bin/cdo -O -s -setgrid,current_grid.txt raw.nc raw_gridded.nc
STDOUT:
STDERR:
# DEBUG - end ===============================================================
RETURNCODE:0


'raw_gridded.nc'

In [13]:
cdo.remapavg('tmp_gridfile.txt', input='raw_gridded.nc', output='regridded_data.nc')

Found method:remapavg
# DEBUG - start =============================================================
CALL  :/usr/local/bin/cdo -O -s -remapavg,tmp_gridfile.txt raw_gridded.nc regridded_data.nc
STDOUT:
STDERR:
# DEBUG - end ===============================================================
RETURNCODE:0


'regridded_data.nc'

In [14]:
ds=xr.open_dataset('regridded_data.nc')

In [15]:
ds

<xarray.Dataset>
Dimensions:  (date: 1, lon: 180, lat: 90)
Coordinates:
  * date     (date) datetime64[ns] 2020-01-01
  * lon      (lon) float64 -179.0 -177.0 -175.0 -173.0 ... 175.0 177.0 179.0
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
Data variables:
    water    (date, lat, lon) float64 ...
    people   (date, lat, lon) float64 ...
Attributes:
    CDI:          Climate Data Interface version 2.2.1 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed Jun 28 03:06:08 2023: cdo -O -s -remapavg,tmp_gridfile....
    CDO:          Climate Data Operators version 2.2.0 (https://mpimet.mpg.de...